In [1]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
import gspread_dataframe as gd
import pygsheets
from datetime import date
import json
import webbrowser


# Package to save datasets to be used in another script for next analysis
from sklearn import datasets

# enableing max columns for dataframes
pd.set_option('display.max_columns', None)

# Importing credentials for Strava's API
from Credentials import StravaCredentials

# Creating date variable
today = date.today().strftime('%B/%d/%Y')

In [4]:
# Pending, this is not working, not extracting StravaCredentials variable:
# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

# In case you need to get the code again uncomment these:
# webbrowser.open(f"https://www.strava.com/oauth/authorize?client_id={data['client_id']}&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all")
# data['code'] = input("From the web broswer enter the code:")

In [6]:
# Getting refresh token
token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()

# Accessing the token json to get refresh token and access token
refresh_token = token['refresh_token']
access_token = token['access_token']

In [7]:
# Getting athlete info
athlete_url = 'https://www.strava.com/api/v3/athlete'
athlete = requests.get(athlete_url+'?access_token='+access_token).json()

# Covnerting json into dataframe and doing some data cleaning
df_athlete = pd.json_normalize(athlete)
df_athlete = df_athlete.T
df_athlete = df_athlete.reset_index()
df_athlete = df_athlete.rename(columns={0:'info'})
df_athlete

,index,info
0,id,58832723
1,username,blindhydra_athlete
2,resource_state,3
3,firstname,Manuel
4,lastname,Elizaldi
5,bio,None
6,city,Austin
7,state,Texas
8,country,United States
9,sex,M


In [8]:
# Setting up url and page
# This API request gives us the list of activities. 
# The table lacks certain details that we will get from another API request
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = token['access_token']
# Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame()
while True:
    # get page of activities from Strava
    print('Getting page number:',page)
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
    print(f'Extraction of page {page} Complete')
    # if no results then exit loop
    if (not r):
        print('Extration Done')
        break
    r = pd.json_normalize(r)
    activities = activities.append(r)
    
    page += 1

clean_activities = activities[['id',
'name',
'distance',
'elapsed_time',
'total_elevation_gain',
'sport_type',
'start_date','achievement_count',
'athlete_count',
'start_latlng',
'end_latlng',
'average_speed',
'max_speed',
'average_temp',
'average_heartrate',
'max_heartrate',
'average_cadence',
'elev_high',
'elev_low']]

Getting page number: 1
Extraction of page 1 Complete
Getting page number: 2
Extraction of page 2 Complete
Getting page number: 3
Extraction of page 3 Complete
Getting page number: 4
Extraction of page 4 Complete
Getting page number: 5
Extraction of page 5 Complete
Extration Done


In [9]:
# Checkpoint so I don't have to run API get request
#table = pd.read_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\StravaAPI\Outputs\clean_activities.csv')

# Creating copy of activitieas dataframe and renaming some columns
general_table = clean_activities.copy()
general_table = general_table.rename(columns={'elapsed_time':'workout_time_sec','sport_type':'workout_type'})
general_table.head()

,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low
0,8771323929,Afternoon Workout,0.0,1399,0.0,Workout,2023-03-24T21:37:49Z,0,1,[],[],0.0,0.0,34.0,143.7,171.0,NaN,NaN,NaN
1,8766877503,Evening Workout,0.0,2299,0.0,Workout,2023-03-23T23:46:28Z,0,1,[],[],0.0,0.0,34.0,133.5,171.0,NaN,NaN,NaN
2,8760859967,Afternoon Workout,0.0,1745,0.0,Workout,2023-03-22T21:41:20Z,0,1,[],[],0.0,0.0,34.0,152.0,184.0,NaN,NaN,NaN
3,8749434921,Afternoon Workout,0.0,3337,0.0,Workout,2023-03-20T22:44:10Z,0,1,[],[],0.0,0.0,24.0,141.3,184.0,NaN,NaN,NaN
4,8726475508,Afternoon Workout,0.0,1303,0.0,Workout,2023-03-16T21:20:10Z,0,1,[],[],0.0,0.0,29.0,139.6,165.0,NaN,NaN,NaN


In [10]:
# Cleaning dataframe and converting units 
# Aprox calories burned during workout calculation
# Formual -> CB = T * (0.6309*H + 0.1988*W + 0.2017*A - 55.0969) / 4.184
# CB ->  is the number of calories burned
# T ->  is the duration of exercise in minutes
# H ->  is your average heart rate in beats per minute
# W -> is your weight in kilograms = 80
# A -> is your age in years
general_table['aprox_calories_burned'] = round((general_table['workout_time_sec']/60) * ((0.6309*general_table['average_heartrate']) + (0.1988*80) + (0.2017*26 - 55.0969)) / 4.184,0)

# from meters to kilometers
general_table[['distance']] = round(general_table['distance']/1000,2)

# from seconds to minutes
general_table['workout_time_min'] = round(general_table['workout_time_sec']/60,2)

# Fix start_date column into the correct format
general_table[["start_date"]] = pd.to_datetime(general_table['start_date']).dt.date

# Changing name of workout type => Workout
general_table['workout_type'] = general_table['workout_type'].replace({'Workout':'Functional-Cardio Workout'})

# Preview
print('Number of recorded workouts:',len(general_table))
general_table.head()

Number of recorded workouts: 652


,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low,aprox_calories_burned,workout_time_min
0,8771323929,Afternoon Workout,0.0,1399,0.0,Functional-Cardio Workout,2023-03-24,0,1,[],[],0.0,0.0,34.0,143.7,171.0,NaN,NaN,NaN,316.0,23.32
1,8766877503,Evening Workout,0.0,2299,0.0,Functional-Cardio Workout,2023-03-23,0,1,[],[],0.0,0.0,34.0,133.5,171.0,NaN,NaN,NaN,460.0,38.32
2,8760859967,Afternoon Workout,0.0,1745,0.0,Functional-Cardio Workout,2023-03-22,0,1,[],[],0.0,0.0,34.0,152.0,184.0,NaN,NaN,NaN,431.0,29.08
3,8749434921,Afternoon Workout,0.0,3337,0.0,Functional-Cardio Workout,2023-03-20,0,1,[],[],0.0,0.0,24.0,141.3,184.0,NaN,NaN,NaN,734.0,55.62
4,8726475508,Afternoon Workout,0.0,1303,0.0,Functional-Cardio Workout,2023-03-16,0,1,[],[],0.0,0.0,29.0,139.6,165.0,NaN,NaN,NaN,281.0,21.72


In [11]:
# Creating additional dataframes for specific activities:
# Running type workouts
running_activities = general_table.loc[general_table['workout_type'].isin(['Run','TrailRun'])]

# Biking type workouts
biking_activities = general_table.loc[general_table['workout_type'].isin(['Ride','MountainBikeRide'])]

# Functional type workouts
functional_activities = general_table.loc[general_table['workout_type'].isin(['Functional-Cardio Workout'])]

In [12]:
# Variables for activities breakdown dataframe 
today_msg = f'Total workouts as of {today}'
total_workouts = len(general_table)

# Breakdown of workout types:
new = [today_msg,total_workouts]
activities_breakdown = general_table['workout_type'].value_counts().rename_axis('Sport').reset_index(name='Count')
activities_breakdown = activities_breakdown.append(pd.Series(new, index=['Sport','Count']), ignore_index=True)
activities_breakdown

,Sport,Count
0,Functional-Cardio Workout,488
1,Run,60
2,WeightTraining,29
3,Ride,17
4,Swim,16
5,Walk,14
6,MountainBikeRide,10
7,Yoga,5
8,Hike,4
9,TrailRun,3


In [21]:
# General statistics
first_recorded_workout = min(general_table['start_date'])
most_recent_workout=max(general_table['start_date'])
average_workout_duration=round(general_table['workout_time_min'].mean(),2)
aprox_average_calories_burned_per_workout=round(general_table['aprox_calories_burned'].mean(),0)
average_distance_ran=round(running_activities['distance'].mean(),0)
average_biking_distance=round(biking_activities['distance'].mean(),0)

# Storing average calories for later analysis comparing calories from garmin, strava and my calculated value
# %store average_calories_burned_per_workout

# Creating dataframe from general statistics variables
# Create the DataFrame
general_stats_df = pd.DataFrame({
    'First Workout':first_recorded_workout,
    'Most Recent Workout': most_recent_workout,
    'Average Workout Duration in Minutes':average_workout_duration,
    'Approximate Average Calories Burned Per Workout':aprox_average_calories_burned_per_workout,
    'Average Distance Ran in Kilometers':average_distance_ran,
    'Average Biking Distance in Kilometers':average_biking_distance
},index=['Info'])

# Transposing dataframe, setting new index and column
general_stats_df = general_stats_df.T
general_stats_df = general_stats_df.reset_index()
general_stats_df = general_stats_df.rename(columns={'index':'Info','Info':'Data'})
general_stats_df

,Info,Data
0,First Workout,2020-05-20
1,Most Recent Workout,2023-03-24
2,Average Workout Duration in Minutes,41.27
3,Approximate Average Calories Burned Per Workout,511
4,Average Distance Ran in Kilometers,7
5,Average Biking Distance in Kilometers,7


In [24]:
# Creating list of ids of these activities to use in the detailed workout api request 
running_id_list = list(running_activities['id'])
biking_id_list = list(biking_activities['id'])
functional_id_list = list(functional_activities['id'])

In [23]:
# Get detailed view of workouts function:
# This function will get the data for each workout, if it reaches the API request limit it will stop the process
# The API rate limit allows us to do 100 requests for each 15 mintues. To prvent passing this limit we only grab -
# - the most recent 100 workouts from each list.
def GetWorkoutData(workout_list):
    workout_info = []
    workout_num = 1
    if len(workout_list)>100:
        print('This workout list is too large, reducing to the 100 most recent workouts.')
        workout_list = workout_list[:100]
        for i in workout_list:
            print('Extracting workout:', workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break
    else:
        for i in workout_list:
            print('Extracting workout:',workout_num)
            req = requests.get(url = f'https://www.strava.com/api/v3/activities/{i}?access_token='+access_token)
            if req.status_code == 200:
                req = req.json()
                workout_info.append(req)
                workout_num += 1
            else:
                print('API Rate limit exceeded, stopping extraction')
                break
    return workout_info

### Using the function to get a json of the detailed data for functional, running and biking workouts
#### run every 15 minutes

In [70]:
running_workouts = GetWorkoutData(running_id_list)

Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9
Extracting workout: 10
Extracting workout: 11
Extracting workout: 12
Extracting workout: 13
Extracting workout: 14
Extracting workout: 15
Extracting workout: 16
Extracting workout: 17
Extracting workout: 18
Extracting workout: 19
Extracting workout: 20
Extracting workout: 21
Extracting workout: 22
Extracting workout: 23
Extracting workout: 24
Extracting workout: 25
Extracting workout: 26
Extracting workout: 27
Extracting workout: 28
Extracting workout: 29
Extracting workout: 30
Extracting workout: 31
Extracting workout: 32
Extracting workout: 33
Extracting workout: 34
Extracting workout: 35
Extracting workout: 36
Extracting workout: 37
Extracting workout: 38
Extracting workout: 39
Extracting workout: 40
Extracting workout: 41
Extracting workout: 42
Extracting workout: 43
Extracting workout: 

In [72]:
biking_workouts = GetWorkoutData(biking_id_list)

Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9
Extracting workout: 10
Extracting workout: 11
Extracting workout: 12
Extracting workout: 13
Extracting workout: 14
Extracting workout: 15
Extracting workout: 16
Extracting workout: 17
Extracting workout: 18
Extracting workout: 19
Extracting workout: 20
Extracting workout: 21
Extracting workout: 22
Extracting workout: 23
Extracting workout: 24
Extracting workout: 25
Extracting workout: 26
Extracting workout: 27


In [26]:
functional_workouts = GetWorkoutData(functional_id_list)

This workout list is too large, reducing to the 100 most recent workouts.
Extracting workout: 1
Extracting workout: 2
Extracting workout: 3
Extracting workout: 4
Extracting workout: 5
Extracting workout: 6
Extracting workout: 7
Extracting workout: 8
Extracting workout: 9
Extracting workout: 10
Extracting workout: 11
Extracting workout: 12
Extracting workout: 13
Extracting workout: 14
Extracting workout: 15
Extracting workout: 16
Extracting workout: 17
Extracting workout: 18
Extracting workout: 19
Extracting workout: 20
Extracting workout: 21
Extracting workout: 22
Extracting workout: 23
Extracting workout: 24
Extracting workout: 25
Extracting workout: 26
Extracting workout: 27
Extracting workout: 28
Extracting workout: 29
Extracting workout: 30
Extracting workout: 31
Extracting workout: 32
Extracting workout: 33
Extracting workout: 34
Extracting workout: 35
Extracting workout: 36
Extracting workout: 37
Extracting workout: 38
Extracting workout: 39
Extracting workout: 40
Extracting work

In [62]:
# This function will parse the workout json, grab the relevant columns, clean the units and create a lap counter for the final dataframe
def CleanWorkoutJson(workout_json):
    df = pd.json_normalize(workout_json)
    df[['distance']] = round(df['distance']/1000,2)
    df['workout_time_min'] = round(df['elapsed_time']/60,2)
    df[["start_date"]] = pd.to_datetime(df['start_date']).dt.date
    df = df.rename(columns={'id':'activity_id'})
    df = df[['activity_id',
               'name',
               'start_date',
               'sport_type',
               'distance',
               'workout_time_min',
               'calories',
               'total_elevation_gain',
               'start_latlng',
               'end_latlng',
               'average_speed',
               'max_speed',
               'average_temp',
               'average_heartrate',
               'max_heartrate']]
    
    workout_laps = pd.json_normalize(workout_json,'laps')
    workout_laps = workout_laps[['activity.id','name','elapsed_time','distance','average_heartrate','max_heartrate','average_speed','max_speed']]
    workout_laps = workout_laps.rename(columns={'activity.id':'activity_id',
                                                'name':'lap',
                                                'elapsed_time':'lap_elapsed_time_min',
                                                'distance':'lap_distance',
                                                'average_heartrate':'lap_average_heartrate',
                                                'max_heartrate':'lap_max_heartrate',
                                                'average_speed':'lap_average_speed',
                                                'max_speed':'lap_max_speed'})
    workout_laps['lap_elapsed_time_min'] = round(workout_laps['lap_elapsed_time_min']/60,2)
    workout_laps['lap_distance'] = round(workout_laps['lap_distance']/1000,2)
    
    lap_counter = workout_laps['activity_id'].value_counts().rename_axis('activity_id').reset_index(name='lap_count')
    
    merged = df.merge(lap_counter, on = 'activity_id')
    merged['lap_count'] = pd.to_numeric(merged['lap_count'])
    return merged

In [64]:
# Function that calculates stats from workout dataframe
def DescribeWorkoutdf(workout_df):
    first_workout = min(workout_df['start_date'])
    last_workout=max(workout_df['start_date'])
    avg_workout_duration=round(workout_df['workout_time_min'].mean(),2)
    avg_calories_burned_per_workout=workout_df['calories'].mean()
    avg_distance=round(workout_df['distance'].mean(),0)
    workout_counter = len(workout_df)
    avg_laps = round(workout_df['lap_count'].mean(),0)

    # Creating dataframe from general statistics variables
    # Create the DataFrame
    grl_stats_df = pd.DataFrame({
        'First Recorded Workout:':first_workout,
        'Most Recent Workout': last_workout,
        'Average Workout Duration in Minutes':avg_workout_duration,
        'Average Calories Burned Per Workout':avg_calories_burned_per_workout,
        'Average Distance in Kilometers':avg_distance,
        'Number of Workouts:': workout_counter,
        'Average Number of Laps':avg_laps
    },index=['Info'])

    # Transposing dataframe, setting new index and column
    grl_stats_df = grl_stats_df.T
    grl_stats_df = grl_stats_df.reset_index()
    grl_stats_df = grl_stats_df.rename(columns={'index':'Info','Info':'Data'})
    return grl_stats_df

In [77]:
# Functional workouts df:
functional_workouts_df = CleanWorkoutJson(functional_workouts)
functional_workouts_df.head(5)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8771323929,Afternoon Workout,2023-03-24,Workout,0.0,23.32,264.0,0,[],[],0.0,0.0,34,143.7,171.0,6
1,8766877503,Evening Workout,2023-03-23,Workout,0.0,38.32,377.0,0,[],[],0.0,0.0,34,133.5,171.0,5
2,8760859967,Afternoon Workout,2023-03-22,Workout,0.0,29.08,361.0,0,[],[],0.0,0.0,34,152.0,184.0,5
3,8749434921,Afternoon Workout,2023-03-20,Workout,0.0,55.62,461.0,0,[],[],0.0,0.0,24,141.3,184.0,6
4,8726475508,Afternoon Workout,2023-03-16,Workout,0.0,21.72,247.0,0,[],[],0.0,0.0,29,139.6,165.0,5


In [67]:
# Functional workouts description:
functional_workouts_desc = DescribeWorkoutdf(functional_workouts_df)
functional_workouts_desc

,Info,Data
0,First Recorded Workout:,2022-10-23
1,Most Recent Workout,2023-03-24
2,Average Workout Duration in Minutes,38.71
3,Average Calories Burned Per Workout,372.447
4,Average Distance in Kilometers,0
5,Number of Workouts:,94
6,Average Number of Laps,5


In [73]:
running_workouts_df = CleanWorkoutJson(running_workouts)
running_workouts_df.head(5)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8633145882,Afternoon Run,2023-02-27,Run,6.09,39.50,477.0,59.0,"[30.38898054510355, -97.73563295602798]","[30.388930588960648, -97.73604970425367]",2.802,5.188,28.0,156.0,169.0,7
1,8594917333,Weighted Run,2023-02-20,Run,6.08,43.68,524.0,91.0,"[30.3889175131917, -97.73592129349709]","[30.38914575241506, -97.73582858964801]",2.324,3.717,31.0,158.2,174.0,7
2,8587667353,Weighted Run,2023-02-19,Run,6.13,41.67,485.0,82.0,"[30.3885461948812, -97.73615305311978]","[30.38905682042241, -97.73590033873916]",2.455,4.272,17.0,145.4,167.0,7
3,8573739485,Afternoon Run,2023-02-16,Run,5.02,33.63,394.0,53.0,"[30.38702990859747, -97.73675520904362]","[30.388092482462525, -97.73673626594245]",2.835,4.323,16.0,150.9,166.0,6
4,8538434119,Morning Trail Run,2023-02-10,TrailRun,6.00,88.95,546.0,346.0,"[30.27398812584579, -97.8255036380142]","[30.27507819235325, -97.82482428476214]",1.632,3.908,19.0,137.6,177.0,5


In [75]:
running_workouts_desc = DescribeWorkoutdf(running_workouts_df)
running_workouts_desc

,Info,Data
0,First Recorded Workout:,2020-05-20
1,Most Recent Workout,2023-02-27
2,Average Workout Duration in Minutes,51.27
3,Average Calories Burned Per Workout,587.703
4,Average Distance in Kilometers,7
5,Number of Workouts:,63
6,Average Number of Laps,6


In [76]:
biking_workouts_df = CleanWorkoutJson(biking_workouts)
biking_workouts_df.head(5)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8715744482,Afternoon Mountain Bike Ride,2023-03-14,MountainBikeRide,19.84,95.30,571.0,260.0,"[30.40146061219275, -97.68447660841048]","[30.40148257277906, -97.68487382680178]",3.962,9.644,22.0,117.6,144.0,3
1,8688907431,Afternoon Mountain Bike Ride,2023-03-09,MountainBikeRide,13.48,75.07,513.0,160.0,"[30.401425156742334, -97.68480601720512]","[30.40150453336537, -97.68491330556571]",3.629,7.426,29.0,125.9,156.0,2
2,8672611987,Afternoon Ride,2023-03-06,Ride,0.00,47.92,133.0,0.0,[],[],0.000,0.000,30.0,80.8,91.0,1
3,8639260773,Afternoon Mountain Bike Ride,2023-02-28,MountainBikeRide,12.71,53.33,457.0,169.0,"[30.401550130918622, -97.6859537512064]","[30.401377798989415, -97.68642079085112]",4.303,9.694,28.0,128.1,157.0,2
4,8600850460,Afternoon Mountain Bike Ride,2023-02-21,MountainBikeRide,14.01,76.98,571.0,187.0,"[30.401328848674893, -97.68461868166924]","[30.40147377178073, -97.68483233638108]",3.725,8.325,31.0,133.2,166.0,2


In [78]:
biking_workouts_desc = DescribeWorkoutdf(biking_workouts_df)
biking_workouts_desc

,Info,Data
0,First Recorded Workout:,2020-05-26
1,Most Recent Workout,2023-03-14
2,Average Workout Duration in Minutes,57.37
3,Average Calories Burned Per Workout,403.778
4,Average Distance in Kilometers,7
5,Number of Workouts:,27
6,Average Number of Laps,2


### Uploading to google drive

In [49]:
# This function uses gspread and pygsheets modules to upload data to google sheets
def WriteToGsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1
    
# Setting up parameters for write_to_gsheet function
service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\Workout-Analysis-API\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'

In [50]:
sheet_name = 'General_Table'
WriteToGsheet(service_file_path, spreadsheet_id,sheet_name,general_table)
general_table.head(1)

,id,name,distance,workout_time_sec,total_elevation_gain,workout_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low,aprox_calories_burned,workout_time_min
0,8771323929,Afternoon Workout,0.0,1399,0.0,Functional-Cardio Workout,2023-03-24,0,1,[],[],0.0,0.0,34.0,143.7,171.0,NaN,NaN,NaN,316.0,23.32


In [53]:
sheet_name = 'Activities_Breakdown'
WriteToGsheet(service_file_path, spreadsheet_id,sheet_name,activities_breakdown)
activities_breakdown

,Sport,Count
0,Functional-Cardio Workout,488
1,Run,60
2,WeightTraining,29
3,Ride,17
4,Swim,16
5,Walk,14
6,MountainBikeRide,10
7,Yoga,5
8,Hike,4
9,TrailRun,3


In [56]:
sheet_name = 'General_Statistics'
WriteToGsheet(service_file_path, spreadsheet_id,sheet_name,general_stats_df)
general_stats_df

,Info,Data
0,First Workout,2020-05-20
1,Most Recent Workout,2023-03-24
2,Average Workout Duration in Minutes,41.27
3,Approximate Average Calories Burned Per Workout,511
4,Average Distance Ran in Kilometers,7
5,Average Biking Distance in Kilometers,7


In [57]:
# Uploading functional workouts to google sheets:
sheet_name = 'Functional_Workouts' 
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,functional_workouts_df)
functional_workouts_df.head(1)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8771323929,Afternoon Workout,2023-03-24,Workout,0.0,23.32,264.0,0,[],[],0.0,0.0,34,143.7,171.0,6


In [69]:
sheet_name = 'Functional_Workouts_Desc'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,functional_workouts_desc)
functional_workouts_desc

,Info,Data
0,First Recorded Workout:,2022-10-23
1,Most Recent Workout,2023-03-24
2,Average Workout Duration in Minutes,38.71
3,Average Calories Burned Per Workout,372.447
4,Average Distance in Kilometers,0
5,Number of Workouts:,94
6,Average Number of Laps,5


In [79]:
sheet_name = 'Running_Workouts'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,running_workouts_df)
running_workouts_df.head(1)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8633145882,Afternoon Run,2023-02-27,Run,6.09,39.5,477.0,59.0,"[30.38898054510355, -97.73563295602798]","[30.388930588960648, -97.73604970425367]",2.802,5.188,28.0,156.0,169.0,7


In [80]:
sheet_name = 'Running_Workouts_Desc'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,running_workouts_desc)
running_workouts_desc

,Info,Data
0,First Recorded Workout:,2020-05-20
1,Most Recent Workout,2023-02-27
2,Average Workout Duration in Minutes,51.27
3,Average Calories Burned Per Workout,587.703
4,Average Distance in Kilometers,7
5,Number of Workouts:,63
6,Average Number of Laps,6


In [82]:
sheet_name = 'Biking_Workouts'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,biking_workouts_df)
biking_workouts_df.head(1)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,lap_count
0,8715744482,Afternoon Mountain Bike Ride,2023-03-14,MountainBikeRide,19.84,95.3,571.0,260.0,"[30.40146061219275, -97.68447660841048]","[30.40148257277906, -97.68487382680178]",3.962,9.644,22.0,117.6,144.0,3


In [83]:
sheet_name = 'Biking_Workouts_Desc'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,biking_workouts_desc)
biking_workouts_desc

,Info,Data
0,First Recorded Workout:,2020-05-26
1,Most Recent Workout,2023-03-14
2,Average Workout Duration in Minutes,57.37
3,Average Calories Burned Per Workout,403.778
4,Average Distance in Kilometers,7
5,Number of Workouts:,27
6,Average Number of Laps,2
